In [ ]:
import socket
import os
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from Crypto.Cipher import AES
from config import AES_KEY, SERVER_IP, UDP_PORT, TCP_PORT, FRAGMENT_SIZE
from tqdm import tqdm
import time

# AES-256 ile dosyayı şifreler ve nonce, tag, ciphertext şeklinde kaydeder
def encrypt_file(input_file, output_file):
    cipher = AES.new(AES_KEY, AES.MODE_EAX)
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    with open(output_file, 'wb') as f:
        f.write(cipher.nonce)
        f.write(tag)
        f.write(ciphertext)

# UDP protokolüyle dosyayı parçalara ayırarak gönderir
def send_udp(encrypted_filename, progress_bar):
    client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    #1 - İlk paket olarak dosya adını gönder
    file_name = os.path.basename(encrypted_filename).encode()
    client.sendto(file_name, (SERVER_IP, UDP_PORT))

    filesize = os.path.getsize(encrypted_filename)
    sent_bytes = 0 

    # Dosya parçalarını tek tek gönder (fragmented UDP)
    with open(encrypted_filename, 'rb') as f:
        progress_console = tqdm(total=filesize, unit='B', unit_scale=True, desc="Gönderiliyor")
        
        while True:
            fragment = f.read(FRAGMENT_SIZE)
            if not fragment:
                break
            client.sendto(fragment, (SERVER_IP, UDP_PORT))
            sent_bytes += len(fragment) 

            # Dinamik hız ayarı 
            if sent_bytes > 50000:
                delay = 0.01
            else:
                delay = 0.001
            time.sleep(delay)
        
            # GUI progress bar'ı güncelle
            progress_gui = int((sent_bytes / filesize) * 100)  # gui'deki progress bar
            progress_bar['value'] = progress_gui
            progress_bar.update() 
            progress_console.update(len(fragment))  # console'daki progress bar
        progress_console.close()

        
    # Tüm parçalar bittikten sonra EOF göndererek aktarımı sonlandır
    client.sendto(b'EOF', (SERVER_IP, UDP_PORT))
    client.close()

    messagebox.showinfo("Bilgi", "UDP ile gönderim tamamlandı.")
    print("UDP ile gönderim tamamlandı.")

# TCP protokolüyle dosyayı gönderme
def send_tcp(encrypted_filename, progress_bar):
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Bağlantı kur ve ilk paket olarak dosya adını gönder
    client.connect((SERVER_IP, TCP_PORT))
    client.send(os.path.basename(encrypted_filename).encode())
    
    filesize = os.path.getsize(encrypted_filename)
    sent_bytes = 0

    # Dosya parçalarını tek tek gönder (TCP akışı)
    with open(encrypted_filename, 'rb') as f:
        progress_console = tqdm(total=filesize, unit='B', unit_scale=True, desc="Gönderiliyor") # outputtaki progress_bar
        while True:
            chunk = f.read(1024)
            if not chunk:
                break
            client.sendall(chunk)
            sent_bytes += len(chunk)
            progress_gui = int((sent_bytes / filesize) * 100) #gui'deki progress bar
            progress_bar['value'] = progress_gui
            progress_bar.update()
            progress_console.update(len(chunk)) # console'daki progress bar
        progress_console.close()
        
    client.close()
    messagebox.showinfo("Bilgi", "TCP ile gönderim tamamlandı.")
    print("TCP ile gönderim tamamlandı.")

# Kullanıcıdan alınan dosyayı şifrele ve protokole göre gönder
def start_transfer(entry, protocol_var, progress_bar):
    filename = entry.get()
    if not filename:
        messagebox.showwarning("Uyarı", "Lütfen bir dosya seçin.")
        return
    encrypted_filename = filename + '.enc'
    encrypt_file(filename, encrypted_filename)
    
    protocol = protocol_var.get()
    if protocol == "UDP":
        send_udp(encrypted_filename, progress_bar)
    else:
        send_tcp(encrypted_filename, progress_bar)

# Dosya seçme penceresini aç ve seçimi GUI'ye yaz
def browse_file(entry):
    filename = filedialog.askopenfilename()
    if filename:
        entry.delete(0, tk.END)
        entry.insert(0, filename)

# GUI arayüzünü başlatır ve kullanıcı etkileşimini yönetir
def main_gui():
    root = tk.Tk()
    root.title("⚡ Hybrid Secure File Transfer ⚡")
    root.geometry("550x350")
    root.configure(bg="#2d2d2d")  # Koyu tema

    # Başlık
    tk.Label(root, text="⚡ Dosya Aktarım Sistemi ⚡", fg="#4caf50", bg="#2d2d2d",
             font=("Helvetica", 16, "bold")).pack(pady=10)

    # Dosya seçimi
    frame = tk.Frame(root, bg="#2d2d2d")
    frame.pack(pady=10)

    tk.Label(frame, text="Gönderilecek Dosya:", fg="white", bg="#2d2d2d",
             font=("Helvetica", 10, "bold")).pack(side="left")
    entry = tk.Entry(frame, width=40, bg="#f0f0f0", font=("Helvetica", 10))
    entry.pack(side="left", padx=5)
    tk.Button(frame, text="Dosya Seç", command=lambda: browse_file(entry),
               bg="#4caf50", fg="white", activebackground="#45a049",
               font=("Helvetica", 10, "bold")).pack(side="left")

    # Çizgi
    ttk.Separator(root, orient='horizontal').pack(fill='x', padx=10, pady=10)

    # Protokol seçimi
    protocol_var = tk.StringVar(value="UDP")
    tk.Label(root, text="Protokol Seçimi:", fg="white", bg="#2d2d2d",
             font=("Helvetica", 10, "bold")).pack()

    radio_frame = tk.Frame(root, bg="#2d2d2d")
    radio_frame.pack()

    udp_button = tk.Radiobutton(radio_frame, text="UDP", variable=protocol_var, value="UDP",
                                bg="#2d2d2d", fg="white", selectcolor="#4caf50",
                                font=("Helvetica", 10, "bold"),
                                activebackground="#4caf50", activeforeground="white",
                                indicatoron=0, width=10, relief="ridge", bd=3)
    udp_button.pack(side="left", padx=10)

    tcp_button = tk.Radiobutton(radio_frame, text="TCP", variable=protocol_var, value="TCP",
                                bg="#2d2d2d", fg="white", selectcolor="#4caf50",
                                font=("Helvetica", 10, "bold"),
                                activebackground="#4caf50", activeforeground="white",
                                indicatoron=0, width=10, relief="ridge", bd=3)
    tcp_button.pack(side="left", padx=10)

    # Çizgi
    ttk.Separator(root, orient='horizontal').pack(fill='x', padx=10, pady=10)

    # Progress bar
    style = ttk.Style()
    style.theme_use('clam')
    style.configure("TProgressbar", thickness=20, troughcolor="#3e3e3e", background="#4caf50")
    progress_bar = ttk.Progressbar(root, orient="horizontal", length=400, mode="determinate", style="TProgressbar")
    progress_bar.pack(pady=20)

    # Gönder Butonu
    tk.Button(root, text="⚡ Şifrele ve Gönder ⚡",
               command=lambda: start_transfer(entry, protocol_var, progress_bar),
               bg="#4caf50", fg="white", activebackground="#45a049",
               font=("Helvetica", 12, "bold"), padx=10, pady=5).pack()

    root.mainloop()

# Uygulamayı başlat
if __name__ == "__main__":
    main_gui()
